## Importing the libraries

In [2]:
# !sudo apt-get install python3-pip
# !sudo python3 -m pip install ipykernel
# !pip3 install pandas

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
from google.colab import drive


#### Function to read the csv files

In [70]:
def read_csv(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            print(row)

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [72]:
# read_csv("./judges_clean.csv")


# Converting the CSV files to Pandas dataframe

In [73]:
df_judges = pd.read_csv("/content/drive/Shareddrives/Precog/Precog/judges_clean.csv")
df_judges.head(100)


,ddl_judge_id,state_code,dist_code,court_no,judge_position,female_judge,start_date,end_date
0,1,1,1,1,chief judicial magistrate,0 nonfemale,20-09-2013,20-02-2014
1,2,1,1,1,chief judicial magistrate,0 nonfemale,31-10-2013,20-02-2014
2,3,1,1,1,chief judicial magistrate,0 nonfemale,21-02-2014,31-05-2016
3,4,1,1,1,chief judicial magistrate,0 nonfemale,01-06-2016,06-06-2016
4,5,1,1,1,chief judicial magistrate,0 nonfemale,06-06-2016,07-07-2018
...,...,...,...,...,...,...,...,...
95,96,1,1,4,civil judge junior division,0 nonfemale,01-03-2007,01-06-2015
96,97,1,1,4,civil judge junior division,0 nonfemale,01-06-2015,31-08-2017
97,98,1,1,4,civil judge junior division,0 nonfemale,04-06-2018,NaN
98,99,1,1,4,civil judge senior division,0 nonfemale,01-03-2007,04-06-2012


In [74]:
df_judges.head(10)
print("The number of unique type of gender of judges in the data : ")
df_judges["female_judge"].unique()


The number of unique type of gender of judges in the data : 


array(['0 nonfemale', '1 female', '-9998 unclear', nan], dtype=object)

In [75]:
print("The count of those types : ")
df_judges["female_judge"].value_counts()


The count of those types : 


0 nonfemale      67540
1 female         27202
-9998 unclear     3735
Name: female_judge, dtype: int64

In [76]:
#Removing the -9998 unclear rows
df_judges=df_judges[df_judges["female_judge"] != "-9998 unclear"]
df_judges["female_judge"].unique()


array(['0 nonfemale', '1 female', nan], dtype=object)

In [77]:
# Converting the strings to numbers
# "0 nonfemale" & to 0
# "1 female" to 1
# df_judges.loc.__setitem__((slice(None),("female_judge","1 female")),0)
df_judges["female_judge"]=(df_judges["female_judge"]=="1 female").astype(int)
df_judges.head(10)
# preprocessed data for finding out female or not

,ddl_judge_id,state_code,dist_code,court_no,judge_position,female_judge,start_date,end_date
0,1,1,1,1,chief judicial magistrate,0,20-09-2013,20-02-2014
1,2,1,1,1,chief judicial magistrate,0,31-10-2013,20-02-2014
2,3,1,1,1,chief judicial magistrate,0,21-02-2014,31-05-2016
3,4,1,1,1,chief judicial magistrate,0,01-06-2016,06-06-2016
4,5,1,1,1,chief judicial magistrate,0,06-06-2016,07-07-2018
5,6,1,1,1,chief judicial magistrate,1,09-07-2018,NaN
6,7,1,1,1,civil judge junior division,0,01-06-2011,09-06-2013
7,8,1,1,1,civil judge junior division,0,06-06-2011,08-06-2014
8,9,1,1,1,civil judge junior division,0,11-06-2012,31-05-2015
9,10,1,1,1,civil judge junior division,0,29-08-2013,08-06-2014


In [78]:
print("Percentage Of Female Judge In India : ")
value=df_judges["female_judge"].value_counts()
male=value[0]  # assuming the unclear gender to be maintaing the same ratio. The national average is given by :-
female=value[1]
national_average=female/(male+female)
print(national_average*100)


Percentage Of Female Judge In India : 
28.71135598408326


In [79]:
print("The count of those types : ") # Trying to find out effect of position on female representation
df_judges["judge_position"].value_counts()
# male=value[0]  # assuming the unclear gender to be maintaing the same ratio. The national average is given by :-
# female=value[1]
# national_average=female/male
# print(national_average)


The count of those types : 


district and sessions court                       19771
chief judicial magistrate                         14224
civil judge senior division                       13438
civil judge junior division                        9726
civil court                                        4645
                                                  ...  
j.m 1st class cum addl. munsif-ii                     1
presiding officer ftc i                               1
v-addl.munsif- iii                                    1
adhoc addl. distt. and sess. judge - ii               1
court no. 5 of j.m 1st class, muzaffarpur east        1
Name: judge_position, Length: 565, dtype: int64

### Checking the cases that were closed within a year

In [80]:
closed_within_year=0
not_closed_within_year=0
df_judges["start_date"]=pd.to_datetime(df_judges["start_date"])
df_judges["end_date"]=pd.to_datetime(df_judges["end_date"])

df_judges.head()
temp_df=(df_judges["end_date"]-df_judges["start_date"])
temp_df=temp_df.dt.days



closed_within_year=temp_df[temp_df<=365].count()
not_closed_within_year=temp_df[temp_df>365].count()


print("Closed within a year : ",closed_within_year)
print("Not closed within a year : ",not_closed_within_year)


Closed within a year :  44120
Not closed within a year :  32932


### Count the number of judges in each state


In [81]:
# Count the number of judges in each state
judgesperstate=df_judges["state_code"].value_counts()
df_judgesperstate=pd.DataFrame(judgesperstate)

df_judgesperstate["State_code"]=df_judgesperstate.index
df_judgesperstate.rename(columns={"state_code":"Number_of_judges"},inplace=True)
df_judgesperstate.reset_index(drop=True,inplace=True)
df_judgesperstate["State_name"]="temp"

print(df_judgesperstate)


    Number_of_judges  State_code State_name
0              21987           1       temp
1              13999          13       temp
2               6440           3       temp
3               5583          23       temp
4               5390           9       temp
5               4949          17       temp
6               4121           8       temp
7               3655          10       temp
8               3492           2       temp
9               3060           4       temp
10              2677          16       temp
11              2568          29       temp
12              2321          14       temp
13              2283           7       temp
14              2034          11       temp
15              1921          22       temp
16              1550           6       temp
17              1457          26       temp
18              1078          15       temp
19              1032          18       temp
20               899          12       temp
21               897           5

In [82]:
# Getting the state codes 

df_statecodes=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")

#storing only on basis of 2018 codes
df_statecodes=df_statecodes[df_statecodes["year"]==2018]
df_statecodes.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
df_statecodes.sort_values(by="state_code",inplace=True)
df_statecodes.reset_index(drop=True,inplace=True)
print(df_statecodes)


    state_code         state_name
0            1        Maharashtra
1            2     Andhra Pradesh
2            3          Karnataka
3            4             Kerala
4            5   Himachal Pradesh
5            6              Assam
6            7          Jharkhand
7            8              Bihar
8            9          Rajasthan
9           10         Tamil Nadu
10          11             Orissa
11          12  Jammu and Kashmir
12          13      Uttar Pradesh
13          14            Haryana
14          15        Uttarakhand
15          16        West Bengal
16          17            Gujarat
17          18       Chhattisgarh
18          19            Mizoram
19          20            Tripura
20          21          Meghalaya
21          22             Punjab
22          23     Madhya Pradesh
23          24             Sikkim
24          25            Manipur
25          26              Delhi
26          27         Chandigarh
27          29          Telangana
28          30

In [83]:
#assigning the state names to the state codes
for i in range(len(df_judgesperstate)):
    df_judgesperstate.loc[i,"State_name"]=df_statecodes.loc[i,"state_name"]

df_judgesperstate.drop("State_code",axis=1,inplace=True)
print(df_judgesperstate)


    Number_of_judges         State_name
0              21987        Maharashtra
1              13999     Andhra Pradesh
2               6440          Karnataka
3               5583             Kerala
4               5390   Himachal Pradesh
5               4949              Assam
6               4121          Jharkhand
7               3655              Bihar
8               3492          Rajasthan
9               3060         Tamil Nadu
10              2677             Orissa
11              2568  Jammu and Kashmir
12              2321      Uttar Pradesh
13              2283            Haryana
14              2034        Uttarakhand
15              1921        West Bengal
16              1550            Gujarat
17              1457       Chhattisgarh
18              1078            Mizoram
19              1032            Tripura
20               899          Meghalaya
21               897             Punjab
22               438     Madhya Pradesh
23               324             Sikkim


**Female Judges Representation In different States over the Years (2014-2019)**

In [84]:
whole_data = pd.read_csv("/content/drive/Shareddrives/Precog/Precog/judges_clean.csv")
# Removing the Unclear data
whole_data=whole_data[whole_data["female_judge"] != "-9998 unclear"]
whole_data["female_judge"]=(whole_data["female_judge"]=="1 female").astype(int)
# whole_data.drop["dist_code","court_no","ddl_judge_id","judge_position","start_date","end_date"]
whole_data.drop(whole_data.columns[[0,2,3,4]],axis=1,inplace=True)
whole_data.head(100)
# whole_data.max()
# whole_data.min()

,state_code,female_judge,start_date,end_date
0,1,0,20-09-2013,20-02-2014
1,1,0,31-10-2013,20-02-2014
2,1,0,21-02-2014,31-05-2016
3,1,0,01-06-2016,06-06-2016
4,1,0,06-06-2016,07-07-2018
...,...,...,...,...
95,1,0,01-03-2007,01-06-2015
96,1,0,01-06-2015,31-08-2017
97,1,0,04-06-2018,NaN
98,1,0,01-03-2007,04-06-2012


In [85]:
data_2014=pd.DataFrame()
data_2014=whole_data[whole_data["end_date"] >= ("01-01-2014")]
data_2014=data_2014[data_2014["start_date"] <= ("01-01-2014")]
data_2015=pd.DataFrame()
data_2015=whole_data[whole_data["end_date"] >= ("01-01-2015")]
data_2015=data_2015[data_2015["start_date"] <= ("01-01-2015")]
data_2016=pd.DataFrame()
data_2016=whole_data[whole_data["end_date"] >= ("01-01-2016")]
data_2016=data_2016[data_2016["start_date"] <= ("01-01-2016")]
data_2017=pd.DataFrame()
data_2017=whole_data[whole_data["end_date"] >= ("01-01-2017")]
data_2017=data_2017[data_2017["start_date"] <= ("01-01-2017")]
data_2018=pd.DataFrame()
data_2018=whole_data[whole_data["end_date"] >= ("01-01-2018")]
data_2018=data_2018[data_2018["start_date"] <= ("01-01-2018")]
data_2019=pd.DataFrame()
data_2019=whole_data[whole_data["end_date"] >= ("01-01-2019")]
data_2019=data_2019[data_2019["start_date"] <= ("01-01-2019")]
# data_2019.count()

In [86]:
# Analysing 2014 female judge represenatation state wise


# data_2014.max()
# state_code              30
# female_judge             1
# start_date      01-01-2014
# end_date        31-12-2015

# data_2014.min()
# state_code               1
# female_judge             0
# start_date      01-01-1995
# end_date        01-01-2015

# data_2014["state_code"].value_counts() 
jdg_female=pd.DataFrame()
jdg_female=data_2014[data_2014["female_judge"] == 1]
temp_2014=jdg_female["state_code"].value_counts()
jdg_2014_f=pd.DataFrame(temp_2014)
jdg_2014_f["State_Code"]=jdg_2014_f.index
jdg_2014_f.rename(columns={"state_code":"Number_of_Female_Judges"},inplace=True)
jdg_2014_f.reset_index(drop=True,inplace=True)
jdg_2014_f.head(10)

,Number_of_Female_Judges,State_Code
0,56,1
1,37,16
2,32,23
3,22,3
4,22,10
5,14,15
6,12,12
7,9,17
8,9,29
9,8,2


In [87]:
jdg_total = data_2014["state_code"].value_counts()
jdg_2014=pd.DataFrame(jdg_total)
jdg_2014["State_Code"]=jdg_2014.index
jdg_2014.rename(columns={"state_code":"Number_of_Judges"},inplace=True)
jdg_2014.reset_index(drop=True,inplace=True)
jdg_2014["State_name"]="temp"
jdg_2014=pd.merge(jdg_2014,jdg_2014_f,how="outer",on="State_Code")
jdg_2014["Number_of_Female_Judges"]=jdg_2014['Number_of_Female_Judges'].fillna(0)
jdg_2014 = jdg_2014.astype({'Number_of_Female_Judges':'int'})
state_code_2014=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2014=state_code_2014[state_code_2014["year"]==2014]
state_code_2014.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2014.sort_values(by="state_code",inplace=True)
state_code_2014.reset_index(drop=True,inplace=True)
for i in range (len(jdg_2014)):
  jdg_2014.loc[i,"State_name"]=state_code_2014.loc[i,"state_name"]

jdg_2014.drop("State_Code",axis=1,inplace=True)
jdg_2014



,Number_of_Judges,State_name,Number_of_Female_Judges
0,231,Maharashtra,56
1,150,Andhra Pradesh,32
2,95,Karnataka,7
3,93,Kerala,37
4,82,Himachal Pradesh,12
5,79,Assam,22
6,54,Jharkhand,22
7,37,Bihar,4
8,27,Rajasthan,9
9,26,Tamil Nadu,2


In [88]:
jdg_female=pd.DataFrame()
jdg_female=data_2015[data_2015["female_judge"] == 1]
temp_2015=jdg_female["state_code"].value_counts()
jdg_2015_f=pd.DataFrame(temp_2015)
jdg_2015_f["State_Code"]=jdg_2015_f.index
jdg_2015_f.rename(columns={"state_code":"Number_of_Female_Judges"},inplace=True)
jdg_2015_f.reset_index(drop=True,inplace=True)
jdg_total = data_2015["state_code"].value_counts()
jdg_2015=pd.DataFrame(jdg_total)
jdg_2015["State_Code"]=jdg_2015.index
jdg_2015.rename(columns={"state_code":"Number_of_Judges"},inplace=True)
jdg_2015.reset_index(drop=True,inplace=True)
jdg_2015["State_name"]="temp"
jdg_2015=pd.merge(jdg_2015,jdg_2015_f,how="outer",on="State_Code")
jdg_2015["Number_of_Female_Judges"]=jdg_2015['Number_of_Female_Judges'].fillna(0)
jdg_2015 = jdg_2015.astype({'Number_of_Female_Judges':'int'})
state_code_2015=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2015=state_code_2015[state_code_2015["year"]==2015]
state_code_2015.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2015.sort_values(by="state_code",inplace=True)
state_code_2015.reset_index(drop=True,inplace=True)
for i in range (len(jdg_2015)):
  jdg_2015.loc[i,"State_name"]=state_code_2015.loc[i,"state_name"]

jdg_2015.drop("State_Code",axis=1,inplace=True)
jdg_2015

,Number_of_Judges,State_name,Number_of_Female_Judges
0,289,Maharashtra,66
1,176,Andhra Pradesh,39
2,153,Karnataka,14
3,94,Kerala,37
4,83,Himachal Pradesh,12
5,79,Assam,22
6,65,Jharkhand,27
7,53,Bihar,26
8,37,Rajasthan,4
9,33,Tamil Nadu,5


In [89]:
jdg_female=pd.DataFrame()
jdg_female=data_2016[data_2016["female_judge"] == 1]
temp_2016=jdg_female["state_code"].value_counts()
jdg_2016_f=pd.DataFrame(temp_2016)
jdg_2016_f["State_Code"]=jdg_2016_f.index
jdg_2016_f.rename(columns={"state_code":"Number_of_Female_Judges"},inplace=True)
jdg_2016_f.reset_index(drop=True,inplace=True)
jdg_total = data_2016["state_code"].value_counts()
jdg_2016=pd.DataFrame(jdg_total)
jdg_2016["State_Code"]=jdg_2016.index
jdg_2016.rename(columns={"state_code":"Number_of_Judges"},inplace=True)
jdg_2016.reset_index(drop=True,inplace=True)
jdg_2016["State_name"]="temp"
jdg_2016=pd.merge(jdg_2016,jdg_2016_f,how="outer",on="State_Code")
jdg_2016["Number_of_Female_Judges"]=jdg_2016['Number_of_Female_Judges'].fillna(0)
jdg_2016 = jdg_2016.astype({'Number_of_Female_Judges':'int'})
state_code_2016=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2016=state_code_2016[state_code_2016["year"]==2016]
state_code_2016.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2016.sort_values(by="state_code",inplace=True)
state_code_2016.reset_index(drop=True,inplace=True)
for i in range (len(jdg_2016)):
  jdg_2016.loc[i,"State_name"]=state_code_2016.loc[i,"state_name"]

jdg_2016.drop("State_Code",axis=1,inplace=True)
jdg_2016

,Number_of_Judges,State_name,Number_of_Female_Judges
0,308,Maharashtra,72
1,176,Andhra Pradesh,39
2,158,Karnataka,14
3,94,Kerala,37
4,86,Himachal Pradesh,12
5,79,Assam,22
6,66,Jharkhand,27
7,59,Bihar,28
8,38,Rajasthan,4
9,33,Tamil Nadu,5


In [90]:
jdg_female=pd.DataFrame()
jdg_female=data_2017[data_2017["female_judge"] == 1]
temp_2017=jdg_female["state_code"].value_counts()
jdg_2017_f=pd.DataFrame(temp_2017)
jdg_2017_f["State_Code"]=jdg_2017_f.index
jdg_2017_f.rename(columns={"state_code":"Number_of_Female_Judges"},inplace=True)
jdg_2017_f.reset_index(drop=True,inplace=True)
jdg_total = data_2017["state_code"].value_counts()
jdg_2017=pd.DataFrame(jdg_total)
jdg_2017["State_Code"]=jdg_2017.index
jdg_2017.rename(columns={"state_code":"Number_of_Judges"},inplace=True)
jdg_2017.reset_index(drop=True,inplace=True)
jdg_2017["State_name"]="temp"
jdg_2017=pd.merge(jdg_2017,jdg_2017_f,how="outer",on="State_Code")
jdg_2017["Number_of_Female_Judges"]=jdg_2017['Number_of_Female_Judges'].fillna(0)
jdg_2017 = jdg_2017.astype({'Number_of_Female_Judges':'int'})
state_code_2017=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2017=state_code_2017[state_code_2017["year"]==2017]
state_code_2017.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2017.sort_values(by="state_code",inplace=True)
state_code_2017.reset_index(drop=True,inplace=True)
for i in range (len(jdg_2017)):
  jdg_2017.loc[i,"State_name"]=state_code_2017.loc[i,"state_name"]

jdg_2017.drop("State_Code",axis=1,inplace=True)
jdg_2017

,Number_of_Judges,State_name,Number_of_Female_Judges
0,316,Maharashtra,72
1,178,Andhra Pradesh,39
2,162,Karnataka,15
3,95,Kerala,37
4,86,Himachal Pradesh,12
5,79,Assam,22
6,69,Jharkhand,27
7,59,Bihar,28
8,38,Rajasthan,4
9,33,Tamil Nadu,11


In [91]:
jdg_female=pd.DataFrame()
jdg_female=data_2018[data_2018["female_judge"] == 1]
temp_2018=jdg_female["state_code"].value_counts()
jdg_2018_f=pd.DataFrame(temp_2018)
jdg_2018_f["State_Code"]=jdg_2018_f.index
jdg_2018_f.rename(columns={"state_code":"Number_of_Female_Judges"},inplace=True)
jdg_2018_f.reset_index(drop=True,inplace=True)
jdg_total = data_2018["state_code"].value_counts()
jdg_2018=pd.DataFrame(jdg_total)
jdg_2018["State_Code"]=jdg_2018.index
jdg_2018.rename(columns={"state_code":"Number_of_Judges"},inplace=True)
jdg_2018.reset_index(drop=True,inplace=True)
jdg_2018["State_name"]="temp"
jdg_2018=pd.merge(jdg_2018,jdg_2018_f,how="outer",on="State_Code")
jdg_2018["Number_of_Female_Judges"]=jdg_2018['Number_of_Female_Judges'].fillna(0)
jdg_2018 = jdg_2018.astype({'Number_of_Female_Judges':'int'})
state_code_2018=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2018=state_code_2018[state_code_2018["year"]==2018]
state_code_2018.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2018.sort_values(by="state_code",inplace=True)
state_code_2018.reset_index(drop=True,inplace=True)
for i in range (len(jdg_2018)):
  jdg_2018.loc[i,"State_name"]=state_code_2018.loc[i,"state_name"]

jdg_2018.drop("State_Code",axis=1,inplace=True)
jdg_2018

,Number_of_Judges,State_name,Number_of_Female_Judges
0,344,Maharashtra,81
1,179,Andhra Pradesh,39
2,172,Karnataka,15
3,95,Kerala,37
4,86,Himachal Pradesh,12
5,81,Assam,22
6,72,Jharkhand,34
7,71,Bihar,27
8,38,Rajasthan,4
9,33,Tamil Nadu,11


In [92]:
jdg_female=pd.DataFrame()
jdg_female=data_2019[data_2019["female_judge"] == 1]
temp_2019=jdg_female["state_code"].value_counts()
jdg_2019_f=pd.DataFrame(temp_2019)
jdg_2019_f["State_Code"]=jdg_2019_f.index
jdg_2019_f.rename(columns={"state_code":"Number_of_Female_Judges"},inplace=True)
jdg_2019_f.reset_index(drop=True,inplace=True)
jdg_total = data_2019["state_code"].value_counts()
jdg_2019=pd.DataFrame(jdg_total)
jdg_2019["State_Code"]=jdg_2019.index
jdg_2019.rename(columns={"state_code":"Number_of_Judges"},inplace=True)
jdg_2019.reset_index(drop=True,inplace=True)
jdg_2019["State_name"]="temp"
jdg_2019=pd.merge(jdg_2019,jdg_2019_f,how="outer",on="State_Code")
jdg_2019["Number_of_Female_Judges"]=jdg_2019['Number_of_Female_Judges'].fillna(0)
jdg_2019 = jdg_2019.astype({'Number_of_Female_Judges':'int'})
state_code_2019=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2019=state_code_2019[state_code_2019["year"]==2018]
state_code_2019.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2019.sort_values(by="state_code",inplace=True)
state_code_2019.reset_index(drop=True,inplace=True)
for i in range (len(jdg_2019)):
  jdg_2019.loc[i,"State_name"]=state_code_2019.loc[i,"state_name"]

jdg_2019.drop("State_Code",axis=1,inplace=True)
jdg_2019

,Number_of_Judges,State_name,Number_of_Female_Judges
0,365,Maharashtra,84
1,184,Andhra Pradesh,17
2,179,Karnataka,39
3,96,Kerala,37
4,86,Himachal Pradesh,12
5,81,Assam,22
6,76,Jharkhand,35
7,71,Bihar,27
8,52,Rajasthan,19
9,48,Tamil Nadu,12


State Wise average Tenure Of Judges

In [25]:
judges=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/judges_clean.csv")
judges.drop(judges.columns[[0,2,3,4,5]],axis=1,inplace=True)
state_code=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code=state_code[state_code["year"]==2018]
state_code.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code.sort_values(by="state_code",inplace=True)
state_code.reset_index(drop=True,inplace=True)
judges=pd.merge(judges,state_code,on="state_code")
judges=judges.dropna()
judges["start_date"]=pd.to_datetime(judges["start_date"],format="%d-%m-%Y",errors='coerce')
judges["end_date"]=pd.to_datetime(judges["end_date"],format="%d-%m-%Y",errors='coerce')
judges["tenure"]=judges["end_date"]-judges["start_date"]
judges=judges.groupby('state_name',as_index=False)["tenure"].mean()
judges

<ipython-input-25-86215d2490e2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judges["start_date"]=pd.to_datetime(judges["start_date"],format="%d-%m-%Y",errors='coerce')
<ipython-input-25-86215d2490e2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  judges["end_date"]=pd.to_datetime(judges["end_date"],format="%d-%m-%Y",errors='coerce')
<ipython-input-25-86215d2490e2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,state_name,tenure
0,Andhra Pradesh,425 days 10:19:48.540245568
1,Assam,544 days 21:29:03.702451392
2,Bihar,380 days 18:29:05.199063232
3,Chandigarh,811 days 02:40:00
4,Chhattisgarh,648 days 19:15:06.955177744
5,Delhi,386 days 16:50:35.897435896
6,Goa,584 days 05:10:41.095890408
7,Gujarat,251 days 03:06:59.906687404
8,Haryana,679 days 13:17:53.759713088
9,Himachal Pradesh,584 days 10:36:05.179856112


Average Case Decision time in States over time

In [132]:
from pandas.core.generic import Axis
cases_2011=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2011.csv")
cases_2011.drop(cases_2011.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2011["date_of_decision"]=pd.to_datetime(cases_2011["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2011["date_of_filing"]=pd.to_datetime(cases_2011["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2011= cases_2011[cases_2011['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2011= cases_2011[cases_2011['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2011["decision_time"]=cases_2011["date_of_decision"]-cases_2011["date_of_filing"]
state_code_2011=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2011=state_code_2011[state_code_2011["year"]==2011]
state_code_2011.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2011.sort_values(by="state_code",inplace=True)
state_code_2011.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2011=pd.merge(cases_2011,state_code_2011,on='state_code')
df_2011=cases_2011.groupby('state_code',as_index=False)["decision_time"].mean()
df_2011=pd.merge(df_2011,state_code_2011,on='state_code')
df_2011

,state_code,decision_time,state_name
0,1,760 days 23:38:09.292223344,Maharashtra
1,2,888 days 15:07:38.284040992,Andhra Pradesh
2,3,564 days 18:43:57.708214104,Karnataka
3,4,878 days 14:41:23.930757120,Kerala
4,5,1021 days 07:55:52.437592144,Himachal Pradesh
5,6,891 days 15:22:58.953635264,Assam
6,7,1410 days 03:19:00.302504128,Jharkhand
7,8,1564 days 16:01:42.073023936,Bihar
8,9,1328 days 06:36:30.331813536,Rajasthan
9,10,1098 days 02:11:10.800418816,Tamil Nadu


In [133]:
from pandas.core.generic import Axis
cases_2012=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2012.csv")
cases_2012.drop(cases_2012.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2012["date_of_decision"]=pd.to_datetime(cases_2012["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2012["date_of_filing"]=pd.to_datetime(cases_2012["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2012= cases_2012[cases_2012['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2012= cases_2012[cases_2012['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2012["decision_time"]=cases_2012["date_of_decision"]-cases_2012["date_of_filing"]
state_code_2012=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2012=state_code_2012[state_code_2012["year"]==2012]
state_code_2012.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2012.sort_values(by="state_code",inplace=True)
state_code_2012.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2012=pd.merge(cases_2012,state_code_2012,on='state_code')
df_2012=cases_2012.groupby('state_code',as_index=False)["decision_time"].mean()
df_2012=pd.merge(df_2012,state_code_2012,on='state_code')
df_2012

,state_code,decision_time,state_name
0,1,704 days 15:04:28.607235760,Maharashtra
1,2,828 days 14:22:21.044737072,Andhra Pradesh
2,3,475 days 12:17:55.245161256,Karnataka
3,4,710 days 23:27:04.291885792,Kerala
4,5,736 days 09:38:36.256818216,Himachal Pradesh
5,6,749 days 21:37:36.720185096,Assam
6,7,997 days 15:40:43.820277440,Jharkhand
7,8,1237 days 15:55:41.259759280,Bihar
8,9,958 days 23:43:42.087412896,Rajasthan
9,10,936 days 21:54:06.249002384,Tamil Nadu


In [134]:
from pandas.core.generic import Axis
cases_2013=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2013.csv")
cases_2013.drop(cases_2013.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2013["date_of_decision"]=pd.to_datetime(cases_2013["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2013["date_of_filing"]=pd.to_datetime(cases_2013["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2013= cases_2013[cases_2013['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2013= cases_2013[cases_2013['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2013["decision_time"]=cases_2013["date_of_decision"]-cases_2013["date_of_filing"]
state_code_2013=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2013=state_code_2013[state_code_2013["year"]==2013]
state_code_2013.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2013.sort_values(by="state_code",inplace=True)
state_code_2013.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2013=pd.merge(cases_2013,state_code_2013,on='state_code')
df_2013=cases_2013.groupby('state_code',as_index=False)["decision_time"].mean()
df_2013=pd.merge(df_2013,state_code_2013,on='state_code')
df_2013

,state_code,decision_time,state_name
0,1,572 days 22:16:54.270397400,Maharashtra
1,2,708 days 17:44:58.294464952,Andhra Pradesh
2,3,373 days 05:07:23.028961804,Karnataka
3,4,590 days 09:39:04.876986864,Kerala
4,5,606 days 15:06:03.416421040,Himachal Pradesh
5,6,522 days 01:59:32.171169400,Assam
6,7,764 days 15:30:23.995647056,Jharkhand
7,8,825 days 22:36:55.256131768,Bihar
8,9,633 days 00:20:04.991318232,Rajasthan
9,10,734 days 18:31:57.652934016,Tamil Nadu


In [3]:
from pandas.core.generic import Axis
cases_2014=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2014.csv")
cases_2014.drop(cases_2014.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2014["date_of_decision"]=pd.to_datetime(cases_2014["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2014["date_of_filing"]=pd.to_datetime(cases_2014["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2014= cases_2014[cases_2014['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2014= cases_2014[cases_2014['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2014["decision_time"]=cases_2014["date_of_decision"]-cases_2014["date_of_filing"]
state_code_2014=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2014=state_code_2014[state_code_2014["year"]==2014]
state_code_2014.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2014.sort_values(by="state_code",inplace=True)
state_code_2014.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2014=pd.merge(cases_2014,state_code_2014,on='state_code')
df_2014=cases_2014.groupby('state_code',as_index=False)["decision_time"].mean()
df_2014=pd.merge(df_2014,state_code_2014,on='state_code')
df_2014

,state_code,decision_time,state_name
0,1,479 days 18:39:58.686739488,Maharashtra
1,2,540 days 20:28:20.226663920,Andhra Pradesh
2,3,299 days 00:23:32.144245552,Karnataka
3,4,455 days 11:46:26.993861080,Kerala
4,5,464 days 03:51:28.201892744,Himachal Pradesh
5,6,471 days 05:28:30.709903592,Assam
6,7,495 days 02:15:26.118704360,Jharkhand
7,8,528 days 00:07:39.859169472,Bihar
8,9,522 days 11:50:33.947813024,Rajasthan
9,10,526 days 23:20:48.099718928,Tamil Nadu


In [4]:
from pandas.core.generic import Axis
cases_2015=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2015.csv")
cases_2015.drop(cases_2015.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2015["date_of_decision"]=pd.to_datetime(cases_2015["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2015["date_of_filing"]=pd.to_datetime(cases_2015["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2015= cases_2015[cases_2015['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2015= cases_2015[cases_2015['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2015["decision_time"]=cases_2015["date_of_decision"]-cases_2015["date_of_filing"]
state_code_2015=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2015=state_code_2015[state_code_2015["year"]==2015]
state_code_2015.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2015.sort_values(by="state_code",inplace=True)
state_code_2015.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2015=pd.merge(cases_2015,state_code_2015,on='state_code')
df_2015=cases_2015.groupby('state_code',as_index=False)["decision_time"].mean()
df_2015=pd.merge(df_2015,state_code_2015,on='state_code')
df_2015

,state_code,decision_time,state_name
0,1,375 days 07:43:16.423195652,Maharashtra
1,2,451 days 19:59:46.369168360,Andhra Pradesh
2,3,250 days 04:56:35.337643,Karnataka
3,4,376 days 02:31:13.880036704,Kerala
4,5,311 days 11:47:47.705625912,Himachal Pradesh
5,6,380 days 10:06:01.889482252,Assam
6,7,358 days 11:53:43.329143612,Jharkhand
7,8,376 days 22:15:31.330314588,Bihar
8,9,423 days 23:33:01.259985512,Rajasthan
9,10,412 days 10:12:22.617643364,Tamil Nadu


In [5]:
from pandas.core.generic import Axis
cases_2016=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2016.csv")
cases_2016.drop(cases_2016.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2016["date_of_decision"]=pd.to_datetime(cases_2016["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2016["date_of_filing"]=pd.to_datetime(cases_2016["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2016= cases_2016[cases_2016['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2016= cases_2016[cases_2016['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2016["decision_time"]=cases_2016["date_of_decision"]-cases_2016["date_of_filing"]
state_code_2016=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2016=state_code_2016[state_code_2016["year"]==2016]
state_code_2016.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2016.sort_values(by="state_code",inplace=True)
state_code_2016.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2016=pd.merge(cases_2016,state_code_2016,on='state_code')
df_2016=cases_2016.groupby('state_code',as_index=False)["decision_time"].mean()
df_2016=pd.merge(df_2016,state_code_2016,on='state_code')
df_2016

,state_code,decision_time,state_name
0,1,264 days 04:11:57.890240164,Maharashtra
1,2,346 days 13:25:23.855303144,Andhra Pradesh
2,3,185 days 13:20:18.326069332,Karnataka
3,4,285 days 21:39:01.789259696,Kerala
4,5,235 days 21:21:13.936850724,Himachal Pradesh
5,6,288 days 22:27:50.050923784,Assam
6,7,254 days 07:32:39.589495096,Jharkhand
7,8,254 days 06:26:46.386184988,Bihar
8,9,308 days 00:01:32.283170424,Rajasthan
9,10,250 days 10:35:26.213382816,Tamil Nadu


In [6]:
from pandas.core.generic import Axis
cases_2017=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2017.csv")
cases_2017.drop(cases_2017.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2017["date_of_decision"]=pd.to_datetime(cases_2017["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2017["date_of_filing"]=pd.to_datetime(cases_2017["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2017= cases_2017[cases_2017['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2017= cases_2017[cases_2017['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2017["decision_time"]=cases_2017["date_of_decision"]-cases_2017["date_of_filing"]
state_code_2017=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2017=state_code_2017[state_code_2017["year"]==2017]
state_code_2017.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2017.sort_values(by="state_code",inplace=True)
state_code_2017.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2017=pd.merge(cases_2017,state_code_2017,on='state_code')
df_2017=cases_2017.groupby('state_code',as_index=False)["decision_time"].mean()
df_2017=pd.merge(df_2017,state_code_2017,on='state_code')
df_2017

,state_code,decision_time,state_name
0,1,166 days 19:06:47.709992702,Maharashtra
1,2,210 days 02:44:02.038176860,Andhra Pradesh
2,3,117 days 17:52:18.389269370,Karnataka
3,4,218 days 09:54:26.891113168,Kerala
4,5,134 days 06:48:09.504667896,Himachal Pradesh
5,6,156 days 05:30:58.961207296,Assam
6,7,135 days 16:52:19.136795904,Jharkhand
7,8,169 days 10:09:36.046197832,Bihar
8,9,114 days 20:15:01.930567858,Rajasthan
9,10,147 days 18:31:18.292828774,Tamil Nadu


In [7]:
from pandas.core.generic import Axis
cases_2018=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/cases_2018.csv")
cases_2018.drop(cases_2018.columns[[0,1,3,4,5,6,7,8,9,10,11,12,13,16,17,18]],axis=1,inplace=True)
cases_2018["date_of_decision"]=pd.to_datetime(cases_2018["date_of_decision"],format="%Y-%m-%d",errors='coerce')
cases_2018["date_of_filing"]=pd.to_datetime(cases_2018["date_of_filing"],format="%Y-%m-%d",errors='coerce')

cases_2018= cases_2018[cases_2018['date_of_decision'].between('1980-01-01', '2023-01-01')]
cases_2018= cases_2018[cases_2018['date_of_filing'].between('1980-01-01', '2023-01-01')]
cases_2018["decision_time"]=cases_2018["date_of_decision"]-cases_2018["date_of_filing"]
state_code_2018=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2018=state_code_2018[state_code_2018["year"]==2018]
state_code_2018.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2018.sort_values(by="state_code",inplace=True)
state_code_2018.reset_index(drop=True,inplace=True)
from pandas.core.reshape.merge import merge
cases_2018=pd.merge(cases_2018,state_code_2018,on='state_code')
df_2018=cases_2018.groupby('state_code',as_index=False)["decision_time"].mean()
df_2018=pd.merge(df_2018,state_code_2018,on='state_code')
df_2018

,state_code,decision_time,state_name
0,1,78 days 02:01:22.952282557,Maharashtra
1,2,99 days 00:45:31.052351272,Andhra Pradesh
2,3,63 days 07:25:43.841777296,Karnataka
3,4,160 days 10:35:41.407997418,Kerala
4,5,36 days 18:57:31.236794131,Himachal Pradesh
5,6,84 days 18:28:17.454162646,Assam
6,7,35 days 13:29:36.655086540,Jharkhand
7,8,69 days 04:06:46.070136511,Bihar
8,9,48 days 06:47:53.512052264,Rajasthan
9,10,60 days 06:24:14.735649912,Tamil Nadu
